In [1]:
import psycopg2
import numpy as np
import os
import json
import requests
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity



In [8]:
load_dotenv()

db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
embedding_api = os.getenv("EMBEDDING_API")
embedding_model = os.getenv("EMBEDDING_MODEL")

# connect to the database
def get_db_connection():
    connection = psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT")
    )
    return connection
# code from embedding to confirm connection to database
from psycopg2 import OperationalError

def create_connection():
    try:
        connection = psycopg2.connect(
            dbname=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        )
        print("Connection to PostgreSQL successful")
        return connection
    except OperationalError as e:
        print(f"The error '{e}' occurred")
        return None
    

connection = create_connection()

Connection to PostgreSQL successful


In [9]:
# create a function for API requests
def embed_text(raw_text):
    # dictionary called 'headers' that specifies that we are doing a JSON request
    headers = {'Content-Type': 'application/json'}
    # use the embedding model variable from earlier to generate a prompt
    payload = {"model": os.getenv("EMBEDDING_MODEL"), 'prompt': raw_text}
    # converts payload into a JSON string and includes the headers to inform the API of the JSON request
    response = requests.post(os.getenv("EMBEDDING_API"),
                             data=json.dumps(payload),
                             headers=headers)
    return response.json().get("embedding")


In [14]:
# create a function to search the database for relevant verses
def search_verses(query):
    # embed the search query
    query_embedding = embed_text(query)
    # retreive verse embeddings from the database
    connection = get_db_connection()
    cursor = connection.cursor()
    cursor.execute("SELECT id, scripture_text, embeddings FROM verses;")
    verses = cursor.fetchall()
    # calculate similarity between query embedding and each verse embedding using cosine similarity
    similarities = []
    for verse in verses:
        verse_id = verse[0]
        verse_text = verse[1]
        verse_embedding = np.array(verse[2])
        similarity = cosine_similarity([query_embedding], [verse_embedding])[0][0]
        similarities.append((similarity, verse_text))
        
        # Parse the embeddings string back to a list
        try:
            verse_embedding = np.array(json.loads(verse_embedding))  # Convert back to NumPy array
        except Exception as e:
            print(f"Error parsing embedding for verse ID {verse_id}: {e}")
            continue
        
    # sort by similarity score and return the most relevant verses
    similarities.sort(reverse=True, key=lambda x: x[0])
    top_verses = similarities[:5] # top 5 most relevant verses
    
    return top_verses

In [15]:
# create a function to take user input and search
def main():
    query = input("Enter a search term or topic: ")
    results = search_verses(query)
    
    print("\nTop relevant verses: \n")
    for score, verse in results:
        print(f"Similarity: {score:.2f}, Verse: {verse}")
if __name__ == "__main__":
    main()

Enter a search term or topic:  i want a scripture about the Godhead


ValueError: could not convert string to float: '[0.8193589,0.65897876,-1.194383,-0.9700293,-1.045747,-0.1326871,-0.76304734,0.41015095,0.54695725,-0.6065159,0.08380902,0.70778006,0.39000034,-0.5283536,1.0388966,-0.024167178,2.2532067,-1.1115413,-0.0621843,-0.32993492,0.23145732,-0.3363461,-0.29021344,0.46956348,-0.013179876,0.6948355,0.26852277,-1.0034978,1.0588869,-0.03546298,0.032890193,0.27816245,-0.64537257,0.727488,-0.18054949,-0.86406696,-0.9027915,0.43579525,-0.6494167,-1.0325963,-1.0277952,-1.157696,2.0117311,-0.028122354,-0.07604927,-0.44345766,-0.863466,-1.3104184,-1.2702129,0.24953264,0.01014249,1.0265505,1.0201648,1.2284547,-0.27658036,-0.3889331,-0.5507182,-0.9762799,-0.7567877,-0.5430787,-1.4202137,0.2860327,-1.5481765,0.37315595,0.19947964,1.9746907,0.12909955,-0.2537161,-0.39619467,-0.43390512,0.1151119,0.34762493,-0.9605437,0.10749906,-1.8277977,0.22094946,0.79506767,-0.02818607,0.09146328,1.7720106,0.27077433,-0.027841909,-1.158226,0.70992357,-0.9286516,0.19892424,-1.